In [1]:
import numpy as np

test_file_path = "../data/day14/test.txt"
input_file_path = "../data/day14/input.txt"


def read_data(path):
  recipe = {}
  with open(path) as file:
    for line in file:
      line = line.strip()
      if line:
        if "->" in line:
          input, output = line.split(" -> ")
          recipe[input] = output
        else:
          template = line
  return template, recipe


In [2]:
template, recipe = read_data(test_file_path)

In [3]:
def transform(input, dict, count = 1):
  output = []
  for combination in zip(input[:-1],input[1:]):
    new_element = dict.get(str.join("", combination))
    output.append(str.join("",[combination[0], new_element]))
  output = str.join("",[*output, input[-1]])
  if count == 1:
    return output
  return transform(output, dict, count -1)

In [4]:
tests = [
    "NCNBCHB",
    "NBCCNBBBCBHCB",
    "NBBBCNCCNBBNBNBBCHBHHBCHB",
    "NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB",
]
for i, test in enumerate(tests, start=1):
    output = transform(template, recipe, i)
    check = u'\u2714' if output == test else u'\u2716'
    print(check, i, output)


✔ 1 NCNBCHB
✔ 2 NBCCNBBBCBHCB
✔ 3 NBBBCNCCNBBNBNBBCHBHHBCHB
✔ 4 NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB


In [5]:
step10 = transform(template, recipe, 10)

In [6]:
def max_minus_min_freq(input):
  freq = {i : input.count(i) for i in set(input)}
  char_freqs = np.array(list(freq.items()))[:,1].astype(np.int64)
  return char_freqs.max() - char_freqs.min()

In [7]:
assert max_minus_min_freq(step10) == 1588

Using the input

In [8]:
template, recipe = read_data(input_file_path)

max_minus_min_freq(transform(template, recipe, 10))

3095

---------

The above method does not scale. As the problem asks for counts but not for the actual string, let's change the algorithm to only count the number of ocurrences of each pair of letters.

In [9]:
def to_dict (template):
  output = {}
  for i, _ in enumerate(template):
    token = template[i:i+2]
    output[token] = output.get(token,0) + 1
  return output

In [10]:
rules = {key: [key[0] + value, value + key[1]]
         for key, value in recipe.items()}
input = to_dict(template)

iterations = 40

def t(tokens, rules):
    output = {}
    for token, count in tokens.items():
        if token in rules:
            for new_token in rules[token]:
                output[new_token] = count + output.get(new_token, 0)
        else:
            output[token] = count + output.get(token, 0)
    return output


transformed_input = input
for _ in range(iterations):
    transformed_input = t(transformed_input, rules)

letter_counters = {}
for key, count in transformed_input.items():
    letter_counters[key[0]] = letter_counters.get(key[0], 0) + count

ar_count = np.array(list(letter_counters.values())).astype(np.int64)
print(f"{ar_count.max() - ar_count.min()} difference between common and least common")


3152788426516 difference between common and least common
